# Segmenting and Clustering Neighborhoods in Toronto
<hr>

## **PART 1**: Grabbing & Cleaning the Data

In [2]:
#installing beautifulsoup and geopy
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0              |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0-py_0

The following packages

In [56]:
#importing necessary libraries
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


In [4]:
#define the url and read the page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content=requests.get(url).text

In [5]:
#define beautifulsoup object to extract information
bsobj=bs(content,'html.parser')

In [6]:
#we are interested in grabbing the table. I will use find_all to grab html table which contains the tag 'table'
tables=bsobj.find_all('table')
#the desired table is the first on in the list:
table=tables[0]

#### Now I will grab each row which starts with <tr> and ends with </tr> and store it in a list
#### I'm removing the new lines characters '\n' and grabbing the content of the cell by selecting the letters that come after "\<td>" and before "<\td>"


In [7]:

#First I will grab the headers
headers=[]
for td in (table.find_all('tr'))[0].find_all('th'):
    headers.append(str(td).replace('\n','')[4:-5])

#Now the rows
rows=[]
for tr in table.find_all('tr'):
    temp=[]
    for td in tr.find_all('td'):
        temp.append(str(td).replace('\n','')[4:-5])
    rows.append(temp)

#remove the first empty row
rows=rows[1:]

#Rename the first header
headers[0]='Postal_Code'

In [8]:
#print the results
print(headers)
print(rows[0:5])

['Postal_Code', 'Borough', 'Neighborhood']
[['M1A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront']]


In [9]:
#Now I will create the dataframe
df=pd.DataFrame(rows,columns=headers)
df.head()

,Postal_Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Now I will do the cleaning

In [10]:
#First drop the unassigned boroughs
df.drop(index=df[df['Borough']=='Not assigned'].index,inplace=True)

In [11]:
#Convert the slashes '/' into commmas in Neighborhood column
df['Neighborhood'].replace(' /',',',regex=True,inplace=True)

In [12]:
#Now I will check if there are duplicates of postal codes and merge any neighborhoods sharing the same postal_code
print(df.count())
print(df['Postal_Code'].unique().size)

Postal_Code     103
Borough         103
Neighborhood    103
dtype: int64
103


#### Note that unique values of the postal_code are the same as the rows in the entire dataframe (103).
#### This means that there aren't duplicate values for postal_code and we don't need to merge.


#### Now I will check if there are any unassigned neighborhoods to give them the same name s their borough

In [13]:
df[df['Neighborhood']=='Not assigned']

,Postal_Code,Borough,Neighborhood


#### there are none so no further work is required. Now I'm done with the cleaning, the next step is to print the shape of the dataframe, but first I will reindex the dataframe to make it neat.

In [14]:
df.reset_index(drop=True,inplace=True)

In [15]:
df.head()

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
print(df.shape)

(103, 3)


<hr>

## **PART 2**: Adding Coordinates to the Dataframe

In [17]:
#read the csv file for coordinates
coodf=pd.read_csv('Geospatial_Coordinates.csv')

In [18]:
#join the two dataframes using the postal code
newdf=pd.merge(df,coodf,left_on='Postal_Code',right_on='Postal Code',how='left')

#drop the extra postal code column
newdf.drop(columns=['Postal Code'],inplace=True)

In [19]:
newdf.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<hr>

## **PART 3**: Exploration & Clustering

In [20]:
#First I will get the boroughs that has 'Toronto' in their names

neighborhoods=newdf[newdf['Borough'].str.contains('Toronto')]

#### Now I will use geopy library to get the coordinates for Toronto, in order to visualize a map with the above boroughs and neighbourhoods.

In [21]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [22]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

#### Foursquare Credentials

In [23]:
#I removed the keys for security
#CLIENT_ID = 'PRIVATE'
#CLIENT_SECRET = 'PRIVATE'
VERSION = '20180605'

#### Now I will use the function to get the nearby venues in order to explore all the postals/neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now I'll get the Toronto venues using the function to get the top 100 avenues close to the postal codes.
#### NOTE: I decided to go with avenues close to postal codes because postal codes are unique and have multiple neighborhoods. The assignment required that multiple neighborhoods be merged into one row under the same postal code.

In [25]:
toronto_venues = getNearbyVenues(names=neighborhoods['Postal_Code'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  );

M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [26]:
toronto_venues.shape
toronto_venues.head()

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### checking the number of venues for each postal code

In [27]:
toronto_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,5,5,5,5,5,5
M4K,43,43,43,43,43,43
M4L,20,20,20,20,20,20
M4M,40,40,40,40,40,40
M4N,3,3,3,3,3,3
M4P,8,8,8,8,8,8
M4R,18,18,18,18,18,18
M4S,33,33,33,33,33,33
M4T,1,1,1,1,1,1


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


#### Now I'm going to analyze the neighborhoods and preprocess the data in order to do the clustering. First I will convert venue categories into dummy variables.

In [37]:
#make dummy variabbles
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#add postal code to the dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 


fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]




#### Now I will calculate the frequency of venues in postal codes

In [39]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.023256,0.000000,0.00,...,0.000000,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
2,M4L,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.050000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.025000,0.000000,0.025000
4,M4N,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
7,M4S,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.058824,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000


In [40]:
num_top_venues = 5

for code in toronto_grouped['Postal Code']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0       Neighborhood   0.2
1  Health Food Store   0.2
2                Pub   0.2
3        Coffee Shop   0.2
4              Trail   0.2


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.09
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----M4L----
                  venue  freq
0  Fast Food Restaurant  0.10
1                  Park  0.10
2      Sushi Restaurant  0.05
3                   Pub  0.05
4               Brewery  0.05


----M4M----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.08
2      Brewery  0.05
3       Bakery  0.05
4    Gastropub  0.05


----M4N----
           venue  freq
0           Park  0.33
1       Bus Line  0.33
2    Swim School  0.33
3        Airport  0.00
4  Movie Theater  0.00


----M4P----
               venue  freq
0                Gym  0.12
1              Hotel  0.12
2     Sandwich Place  0.12
3   Departme

#### Here, I will be extracting the 10 most common venues for each postal code of toronto

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_venues_sorted = pd.DataFrame(columns=columns)
postal_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    postal_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postal_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Coffee Shop,Pub,Trail,Neighborhood,Health Food Store,Yoga Studio,Distribution Center,Dessert Shop,Diner,Discount Store
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,Park,Fast Food Restaurant,Coffee Shop,Light Rail Station,Sandwich Place,Liquor Store,Burrito Place,Italian Restaurant,Restaurant,Ice Cream Shop
3,M4M,Café,Coffee Shop,American Restaurant,Brewery,Bakery,Gastropub,Yoga Studio,Diner,Bookstore,Seafood Restaurant
4,M4N,Park,Bus Line,Swim School,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Now I will move on to clustering

In [106]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 4, 2, 4, 4, 4, 1, 4], dtype=int32)

In [108]:
#postal_venues_sorted.drop(columns=['Cluster Labels'],axis=1,inplace=True)
postal_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [109]:
toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postal_venues_sorted.set_index('Postal Code'), on='Postal_Code')


In [110]:
toronto_merged.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Restaurant,Beer Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,College Auditorium
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Coffee Shop,Clothing Store,Café,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Bookstore
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Cocktail Bar,Gastropub,Italian Restaurant,American Restaurant,Department Store,Farmers Market,Hotel,Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Trail,Neighborhood,Health Food Store,Yoga Studio,Distribution Center,Dessert Shop,Diner,Discount Store


In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal_Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Observations

In [121]:
#Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,0,Coffee Shop,Pub,Trail,Neighborhood,Health Food Store,Yoga Studio,Distribution Center,Dessert Shop,Diner,Discount Store
68,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
91,Downtown Toronto,0,Park,Playground,Trail,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [125]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Playground,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [126]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Park,Bus Line,Swim School,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


In [127]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Ice Cream Shop,Garden,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [128]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Theater,Café,Yoga Studio,Restaurant,Beer Store
4,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,College Auditorium
9,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Bookstore
15,Downtown Toronto,4,Café,Coffee Shop,Cocktail Bar,Gastropub,Italian Restaurant,American Restaurant,Department Store,Farmers Market,Hotel,Restaurant
20,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Café,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Restaurant,Portuguese Restaurant
24,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop,Bar,Thai Restaurant,Salad Place,Burger Joint,Bubble Tea Shop
25,Downtown Toronto,4,Grocery Store,Café,Park,Baby Store,Candy Store,Restaurant,Italian Restaurant,Diner,Coffee Shop,Nightclub
30,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,American Restaurant,Hotel,Thai Restaurant,Clothing Store,Sushi Restaurant
31,West Toronto,4,Pharmacy,Bakery,Park,Liquor Store,Café,Middle Eastern Restaurant,Bar,Bank,Brewery,Pool
36,Downtown Toronto,4,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Brewery,Sporting Goods Shop,Restaurant,Fried Chicken Joint,Italian Restaurant


#### It seems that cluster 4 contains the postal codes which has cafes/bakery/resturants as popular avenues, while other clusters have stores/playgrounds/etc. as popular avenues.